In [ ]:
import pandas as pd
from google.cloud import bigquery

# Initialize BigQuery client
client = bigquery.Client()

# Define your table path
sentiment_clean = "sentiment-lewagon.sentiment_db.sentiment_clean"
core = "sentiment-lewagon.sentiment_db.core_v3"

# Load the table into a DataFrame
sc = client.list_rows(sentiment_clean).to_dataframe()
sc.rename(columns={'bucket_filepath': 'bucket_file_path'}, inplace=True)
core = client.list_rows(core).to_dataframe()

#### CREATE FINAL TABLE
final_table = pd.merge(core, sc, on = "bucket_file_path", how = "left")

#### push final table to biq query

def upload_fina_table_to_bq_(df):
    """
    Uploads final static table to BQ
    """
    # Rename columns to lowercase
    df.columns = df.columns.str.lower()


    try:
        # Keep only the required columns

        BQ_PROJECT_ID = 'sentiment-lewagon'
        BQ_DATASET_ID = 'sentiment_db'
        BQ_TABLE_ID = 'final_v5'
        table_ref = f"{BQ_PROJECT_ID}.{BQ_DATASET_ID}.{BQ_TABLE_ID}"

        client = bigquery.Client()

        job_config = bigquery.LoadJobConfig(
            write_disposition="WRITE_APPEND"
        )

        job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
        job.result()

        print(f"✅ Uploaded {job.output_rows} rows to {table_ref}")

    except Exception as e:
        print(f"❌ Failed to upload DataFrame to BigQuery: {e}")


###### call function to upload to BQ

upload_fina_table_to_bq_(final_table)
